In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "AhmedBou/Arabic-Meta-Llama-3.1-8B_LoRA", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "قم بصياغة الجملة الإنجليزية التالية باللغة العربية.", # instruction
        "We hope that the last cases will soon be resolved through the mechanisms established for this purpose.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

ونأمل في أن تتم في غضون قريب حلول هذه الحالات الأخيرة عن طريق الآليات المقررة لهذا الغرض.<|end_of_text|>


In [ ]:
def parse_arabic_response(text):
  start_marker = "### Response:\n"
  end_marker = "<|end_of_text|>"
  start_index = text.find(start_marker) + len(start_marker)
  end_index = text.find(end_marker)
  return text[start_index:end_index].strip()

In [ ]:
def generate_arabic(instruction, input_text):
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          instruction, # instruction
          input_text, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  model_output = model.generate(**inputs, max_new_tokens = 128)
  output = tokenizer.decode(model_output[0])
  return parse_arabic_response(output)

## Finetuned model Inference

In [ ]:
from datasets import load_dataset
dataset = load_dataset("AhmedBou/Arabic_instruction_dataset_for_llm_ft", split = "eval")

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# convert the dataset to dataframe
eval_df = dataset.to_pandas()

In [ ]:
eval_df.head()

,input,output,instruction
0,١٥ - وأكد الممثلون أن اﻵليات الوطنية، لكي يتسن...,15. Representatives emphasized that in order t...,Translate the Arabic text below to English.
1,ب إ 3-74 تتصل الزيادة البالغة 500 370 دولار با...,"IS3.74 The increase, in the amount of $370,500...",قم بصياغة الجملة العربية التالية باللغة الإنجل...
2,92 - الرئيس: قال إنه سبق أن تم معالجته على نحو...,92. The Chairman said that it had already been...,قم بتحويل الجملة العربية أدناه إلى الإنجليزية.
3,٢٧ - ومضى المتحدث إلى القول إن حفظ السلم واﻷمن...,27. The maintenance of peace and security in t...,Translate the following to English.
4,فقد لاحظت اللجنة أن إجراء طلب العروض &quot; ير...,The Committee noted that the requests for prop...,Rephrase the following Arabic statement in Eng...


In [ ]:
# Take only the first 100 instruction for this tutorial
eval_df_first_100 = eval_df.head(100)

In [ ]:
%%time
eval_df_first_100['generated_output'] = eval_df_first_100.apply(lambda x: generate_arabic(x['instruction'], x['input']), axis=1)

CPU times: user 6min 30s, sys: 2.5 s, total: 6min 33s
Wall time: 7min 2s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
eval_df_first_100.sample(10)

,input,output,instruction,generated_output
81,4.7 The State party submits that the question ...,4-7 وتدفع الدولة الطرف بأن اللجنة لم تتناول مب...,Rephrase the following English statement in Ar...,4-7 وتقدم الدولة الطرف بيانا بأن مسألة التأثير...
60,&quot; 13 - ترحب بالإسهامات السابقة والجارية ا...,&quot; 13. Welcomes the previous and ongoing c...,يرجى ترجمة الجملة العربية المعطاة إلى الإنجليزية.,&quot; 13. Welcomes the contributions to date ...
14,It required the application of International A...,وتتطلب تطبيق المعايير الدولية للمحاسبة في الإب...,قم بترجمة النص التالي إلى اللغة العربية.,وقد طُلب تطبيق معايير المحاسبة الدولية (IAS) ل...
46,&apos; 3` اتخاذ مناهج وأساليب محورها المتعلم، ...,(iii) Adopt learner-centred methods and approa...,قم بتحويل العبارة العربية المعطاة إلى الإنجليزية.,iii. Pedagogical approaches and methods centre...
29,Their very nature means that these centres are...,إن الطبيعة الذاتية لهذه المراكز تعني أنها مدعو...,قم بتغيير العبارة الإنجليزية التالية إلى اللغة...,وهذه المراكز، من طبيعتها، تدعو إلى دعم جهود ال...
89,"In the judgment in Arrest Warrant, the Interna...",وجاء في الحكم المتعلق بدعوى أمر القبض الصادر ع...,عبر عن الجملة الإنجليزية باللغة العربية.,وفي حكم إحالة طلب الاعتقال، يقول محكمة العدل ا...
49,وسيُنفّذ أحد أنشطة تعزيز القدرات المؤسسية لحما...,One activity on strengthening institutional ca...,قم بصياغة الجملة العربية التالية باللغة الإنجل...,One of the capacity-building activities for in...
82,b. Progress was generally on target in August ...,(ب) كان التقدم يتحقق عموماً حسب المقرر في آب/أ...,قم بترجمة النص التالي إلى اللغة العربية.,باء - كانت التقدم في العموم على المسار المحدد ...
74,وكبديل عن ذلك، يمكن تمويل هذه النفقات من التدف...,"Alternatively, those expenses can be funded ou...",Turn the Arabic sentence below into English.,"As an alternative, such expenditures could be ..."
86,15 - تطلب مرة أخرى إلى الأمين العام أن يتخذ ال...,Requests once again the Secretary-General to t...,عبر عن الجملة العربية باللغة الإنجليزية.,15. Reaffirms that the Secretary-General shoul...


In [ ]:
eval_df_first_100.to_excel('evaluation_file.xlsx', index=False)